In [1]:
from setup import init_django
init_django()

In [2]:
from helpers import CoinMarketCap as cmc

In [3]:
url = 'https://coinmarketcap.com/currencies/duko/'
coin_url = cmc.get_coin_url('AAVE')
html = cmc.scrape(coin_url)

Connecting to scraping browser
Connected! Navigated to https://coinmarketcap.com/currencies/aave
Done Scraping!


In [4]:
from bs4 import BeautifulSoup

In [5]:
def convert_to_number(string):
    """
    Converts a string including symbols and commas to a numerical value.
    """
    number = string.split()[0]
    number = number.replace('$', '').replace(',', '').replace('%', '')
    return float(number)

In [6]:
def get_coin_data(html):
    soup = BeautifulSoup(html)
    
    coin_stats_div = soup.find('div', {'data-module-name': 'Coin-stats'})
    coin_data = {}
    
    if coin_stats_div:
        # Extract price
        price = coin_stats_div.find('span', {'class': 'sc-d1ede7e3-0 fsQm base-text'}).text.strip()
        price = convert_to_number(price)
        coin_data['price'] = price
    
        # Extract price change
        price_change_element = coin_stats_div.find('p', {'class': 'sc-71024e3e-0 sc-58c82cf9-1 ihXFUo iPawMI'})
        price_change_color = price_change_element.get('color')
        price_change_value = price_change_element.text.strip().split('%')[0]
        if price_change_color == 'red':
            price_change_value = '-' + price_change_value
    
        coin_data['price_change'] = float(price_change_value)
    
        # Extract market cap
        market_cap_element = coin_stats_div.find('div', string='Market cap').parent.parent.find('dd')
        market_cap_value = market_cap_element.contents[1].strip()
        market_cap_value = convert_to_number(market_cap_value)
        coin_data['market_cap'] = market_cap_value

        # Extract volume (24h)
        volume_24h_element = coin_stats_div.find('div', string='Volume (24h)').parent.parent.find('dd')
        volume_24h_value = volume_24h_element.contents[1].strip()
        volume_24h_value = convert_to_number(volume_24h_value)
        coin_data['volume'] = volume_24h_value
    
        # Extract Market cap and volume rank
        rank_elements = coin_stats_div.find_all('span', {'class': 'text slider-value rank-value'})
        market_cap_rank_value = int(rank_elements[0].text[1:])
        volume_rank_value = int(rank_elements[1].text[1:])
        coin_data['market_cap_rank'] = market_cap_rank_value
        coin_data['volume_rank'] = volume_rank_value

        # Extract Volume Change
        volume_change_element = coin_stats_div.select_one('div.sc-d1ede7e3-0.sc-cd4f73ae-0.bwRagp.iWXelA.flexBetween:has(dt:-soup-contains("Volume/Market cap (24h)")) dd')
        volume_change_value = convert_to_number(volume_change_element.text.strip())
        coin_data['volume_change'] = volume_change_value
    
        # Extract circulating supply
        circulating_supply_element = coin_stats_div.find('div', string='Circulating supply').parent.parent.find('dd')
        circulating_supply_value = convert_to_number(circulating_supply_element.text.strip())
        coin_data['circulating_supply'] = circulating_supply_value
    
        # Extract total supply
        total_supply_element = coin_stats_div.select_one('div.sc-d1ede7e3-0.sc-cd4f73ae-0.bwRagp.iWXelA.flexBetween:has(dt:-soup-contains("Total supply")) dd')
        total_supply_value = convert_to_number(total_supply_element.text.strip())
        coin_data['total_supply'] = total_supply_value
    
        # Extract Fully diluted market cap
        diluted_market_cap_element = coin_stats_div.select_one('div.sc-d1ede7e3-0.sc-cd4f73ae-0.bwRagp.iWXelA.flexBetween:has(dt:-soup-contains("Fully diluted market cap")) dd')
        diluted_market_cap_value = convert_to_number(diluted_market_cap_element.text.strip())
        coin_data['diluted_market_cap'] = diluted_market_cap_value

        contract = {}
        coin_data['contracts'] = []

        contracts_elements = coin_stats_div.find_all('div', {'class': 'sc-d1ede7e3-0 sc-7f0f401-0 sc-96368265-0 bwRagp gQoblf eBvtSa flexStart'})
        for element in contracts_elements:
            contract_name = element.find('span', {'class': 'sc-71024e3e-0 dEZnuB'}).text.strip()
            address_element = element.find('a')
            contract = {}
            if address_element:
                address_value = address_element.get('href').split('/')[-1]
                contract['name'] = contract_name
                contract['address'] = address_value
                coin_data['contracts'].append(contract)
        # Extract official links
        official_links_element = coin_stats_div.select('.sc-d1ede7e3-0.jTYLCR:has(span:-soup-contains("Official links")) .sc-d1ede7e3-0.sc-7f0f401-0.gRSwoF.gQoblf')
        
        official_links = []
        
        for div in official_links_element:
            link_element = div.find('a')
            if link_element:
                link_dict = {}
                link = link_element.get('href')
                link_text = link_element.text.strip()
                if link and link_text:
                    link_dict['name'] = link_text
                    link_dict['link'] = link
                    official_links.append(link_dict)

        coin_data['official_links'] = official_links
    
        #Extract social links
        social_links_element = coin_stats_div.select('.sc-d1ede7e3-0.jTYLCR:has(span:-soup-contains("Socials")) .sc-d1ede7e3-0.sc-7f0f401-0.gRSwoF.gQoblf')
        
        social_links = []
        for div in social_links_element:
            link_element = div.find('a')
            if link_element:
                link_dict = {}
                link = link_element.get('href')
                link_text = link_element.text.strip()
                if link and link_text:
                    link_dict['name'] = link_text
                    link_dict['link'] = link
                    social_links.append(link_dict)
        coin_data['socials'] = social_links
    else:
        coin_data['error'] = 'Coin statistics div not found in HTML.'
    return coin_data

In [7]:
data = get_coin_data(html)
print(data)

{'price': 102.17, 'price_change': -1.86, 'market_cap': 1516824870.0, 'volume': 73536390.0, 'market_cap_rank': 66, 'volume_rank': 97, 'volume_change': 4.85, 'circulating_supply': 14845774.0, 'total_supply': 16000000.0, 'diluted_market_cap': 1633681687.0, 'contracts': [{'name': 'Ethereum:', 'address': '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9'}], 'official_links': [{'name': 'Whitepaper', 'link': 'https://github.com/aave/aave-protocol/blob/master/docs/Aave_Protocol_Whitepaper_v1_0.pdf'}, {'name': 'GitHub', 'link': 'https://github.com/aave'}], 'socials': [{'name': '𝕏Twitter', 'link': 'https://twitter.com/aave'}, {'name': 'Reddit', 'link': 'https://reddit.com/r/Aave_Official'}, {'name': 'Telegram', 'link': 'https://t.me/Aavesome'}, {'name': 'Chat', 'link': 'https://aave.com/discord'}]}
